# Algorithms for Data Science

## Computing Moments of a Stream

### 1. Preliminaries 

The objective of this lab is to implement the Alon-Matias-Szegedy approach to estimate the $2$nd moment of the stream in which $N$ distinct items from $0$ to $N-1$ appear.

In [17]:
import random

#parameters
N = 256 #N distinct values between 0 and N-1
stream_size = 10000

### 2. Alon-Matias-Szegedy for Second Moments

We implement here the AMS approach when the stream size is known:
1. We choose a number $t$ between $0$ and $\text{stream\_size}-1$ from which the counts are kept
2. When the stream is at timestamp $t$, we initialize $\text{v}=S(t)$ and $c=1$
3. Whenever we encounter $v$ in the stream, we increment $c$ by $1$

At the end of the stream, we output the estimator $\text{stream\_size}\times(2c-1)$

This can be easily extended to an arbirary number of counts, by generating $k$ different timestamps and keeping arrays of $v$ and $c$.

In [18]:
#initialize values and counts
v = []
c = []
#keeping the true counts 
counts = {}
#choosing k timestamps
k = 10
t = []
for _ in range(k): #estimators
  t.append(random.randrange(stream_size))
  v.append(-1)
  c.append(0)

for i in range(stream_size):
  #take a random value between 0 and N-1
  s = random.randrange(N) # stream
  #AMS approach
  for j in range(k):
    if i==t[j]: #chosen timestamp
      v[j] = s
      c[j] = 1
    elif i>t[j] and s==v[j]: #after timestamp
      c[j] += 1
  #true counts (only for evaluation!)
  if s not in counts:
    counts[s] = 0
  counts[s] = counts[s]+1

#true 2nd moment
true = 0
for x in counts.keys():
  true += counts[x]*counts[x]

#2nd moment estimator
est = 0
for x in range(k):
  est += 2*c[x]-1
est = int((stream_size/k)*est)

print('Estimation of 2nd moment: %d'%est)
print('True second moment: %d'%true)

Estimation of 2nd moment: 436000
True second moment: 401098


### 3. **TASK** AMS for Infinite Streams

Implement the case when the estimator does not know the size of the stream.

In this case, instead of generating $k$ timestamps, we proceed to use _Reservoir Sampling_ as explained in the lecture:
1. initialize $v$ and $c$ with the corresponding values in the first $k$ items in the stream $S$,
2. for timestamp $t>k$, we decide whether to replace a $v$ with probability $k/t$,
3. if true, we replace a value (and its corresponding count) at random in the arrays $v$ and $c$ (and re-initialize the values).

In [19]:
# reservoire sampling

#Reservoir Sampling Class Implementation

#k = 10 # initial number of estimators
"""
for i in range(100):
    s = random.randrange(N)
    #we want to keep a sample of the stream, of k values in the array v
    if i<k: #if the timestamp is between 0 and k-1
        print ('Timestamp %d<%d value %d -- fill'%(i,k,s))
        v.append(s) #fill the reservoir
    else:
        #decide whether the new value replaces one of the old ones
        c = random.random() #chooses a value in [0,1] uniformly
        print ('Timestamp %d>=%d value %d -- decide whether to replace'%(i,k,s))
        if c <= k/i: #if c is between [0,k/i] then we want to replace, if c is in (k/i,1] we discard
            #we keep
            j = random.randrange(k) #choose a value between 0 and k-1 to replace
            print ('\t coin %f <= %f -- replace pos %d'%(c,k/i,j))
            v[j] = s #replace with the new value in the stream
        else:
            print ('\t coin %f > %f -- do not replace'%(c,k/i))
    print(v)
"""


class Reservoir_sampling:
    def __init__(self,size):
        self.max_size=size
        self.size = 0
        self.stream_number = 0
        self.reservoir = []
        
        self.real_moment_count = {}
        
    def insert(self,element):
        
        self.stream_number+=1
        
        if self.size<self.max_size: #if the timestamp is between 0 and k-1
            #print ('Timestamp %d<%d value %d -- fill'%(i,k,s))
            self.reservoir.append(element) #fill the reservoir
            self.size+=1
        else:
            #decide whether the new value replaces one of the old ones
            c = random.random() #chooses a value in [0,1] uniformly
            #print ('Timestamp %d>=%d value %d -- decide whether to replace'%(i,k,s))
            if c <= self.max_size/self.stream_number: #if c is between [0,k/i] then we want to replace, if c is in (k/i,1] we discard
                #we keep
                j = random.randrange(self.max_size) #choose a value between 0 and k-1 to replace
                #print ('\t coin %f <= %f -- replace pos %d'%(c,k/i,j))
                self.reservoir[j] = element #replace with the new value in the stream
        
        #compute the real second moment
        if element not in self.real_moment_count:
            self.real_moment_count[element] = 0
        self.real_moment_count[element] = self.real_moment_count[element]+1

    
    def get_reservoir(self):
        return self.reservoir
    
    def get_real_second_moment(self):
        moment = 0
        for x in self.real_moment_count.keys():
            moment += self.real_moment_count[x]*self.real_moment_count[x]
        return moment
        

def second_moment_estimator(reservoir,nb_estimators,stream_size):
    v = []
    c = []
    t = []
    for _ in range(nb_estimators): #estimators
        random_spot = random.randrange(len(reservoir))
        if random_spot not in t:
            t.append(random_spot) # choose arbitrary random spots (index in reservoir)
            v.append(-1)
            c.append(0)
        else:
            #print("collision of estimators spots -1 less estimator")
            
        
    for index in range(len(reservoir)): # for each element in reservoir
        for estimator_index in range(len(t)): # for each estimator
            if(index==t[estimator_index]): #initialization case
                v[estimator_index] = reservoir[index]
                c[estimator_index] = 1
            else:
                if(index>t[estimator_index]) and (v[estimator_index]==reservoir[index]):
                    c[estimator_index] +=1
    est = 0
    for x in range(len(c)):
        est += 2*c[x]-1
    est = int((stream_size/len(c))*est) 
    
    # len(c),len(t) or len(v) is the real number of used estimators while removing redundent ones
    
    #print('Estimation of 2nd moment: %d'%est)
    
    return est
        
    

In [24]:
k = 30 # initial number of estimators
reservoir_size = 300 # size of the reservoire

reservoir_sampler = Reservoir_sampling(reservoir_size) # size of the reservoire

for i in range(stream_size): # stream size is in the start
    reservoir_sampler.insert(random.randrange(N))
    
print(reservoir_sampler.get_reservoir())

assert(reservoir_sampler.stream_number==stream_size)

print("real second moment %d"%reservoir_sampler.get_real_second_moment())
print("estimated second moment %d"%second_moment_estimator(reservoir_sampler.get_reservoir(),k,reservoir_sampler.stream_number))

[101, 226, 240, 51, 112, 76, 114, 112, 100, 14, 125, 99, 97, 99, 167, 120, 51, 22, 110, 198, 123, 92, 139, 57, 145, 186, 135, 77, 115, 155, 174, 81, 10, 0, 32, 195, 135, 169, 18, 17, 117, 39, 233, 117, 28, 29, 193, 211, 17, 180, 214, 36, 80, 146, 177, 31, 165, 6, 248, 94, 153, 135, 241, 153, 27, 242, 221, 83, 159, 94, 102, 239, 112, 203, 143, 204, 184, 95, 25, 241, 41, 237, 250, 209, 186, 84, 62, 7, 150, 16, 213, 89, 123, 124, 93, 29, 8, 30, 166, 146, 231, 54, 31, 80, 69, 0, 99, 232, 39, 218, 247, 79, 128, 115, 116, 7, 123, 196, 111, 143, 73, 50, 101, 91, 174, 134, 95, 19, 26, 50, 229, 183, 2, 75, 41, 89, 141, 84, 95, 4, 101, 129, 187, 104, 134, 116, 254, 235, 24, 176, 201, 17, 134, 123, 210, 137, 205, 199, 139, 93, 246, 37, 39, 18, 109, 101, 226, 80, 78, 5, 50, 55, 109, 223, 5, 214, 30, 184, 186, 146, 170, 27, 4, 79, 14, 2, 32, 186, 140, 76, 240, 184, 220, 131, 189, 20, 170, 196, 40, 145, 82, 64, 175, 127, 196, 236, 242, 175, 44, 94, 53, 227, 133, 116, 3, 61, 82, 219, 157, 21, 163, 12

_You can use this cell to write your discussion of the results_